### Intro
Linear score functiion $s = W x$, where W is the weight matrix and x is a signle input(an image of CIFAR-10), a column vector of size [3072 X 1] and W is of size [10 X 3073] where number of class levels is 10.

Neual network would instead compute $s = W_2 \max(0, W_1 x)$. W1
 could be, for example, a [100x3072] matrix transforming the image into a 100-dimensional intermediate vector. The function max(0,−)  is a non-linearity that is applied elementwise.W2
  would then be of size [10x100], so that we again get 10 numbers out that we interpret as the class scores.

A three-layer neural network could analogously look like $s = W_3 \max(0, W_2 \max(0, W_1 x))$ where all of W3,W2,W1 are parameters to be learned. 



### Biological Motivation
Inputs(dendrites)-->inputs multiplied with weights(synapses)-->neurons(sum of all inputs)-->activation function(fire)-->output

An example code for forward-propagating a single neuron might look as follows:

In [ ]:
import numpy as np
import math
class Neurons(object):
    def forward(self,inputs):
        """assume inputs and weights are 1-D numpy arrays and bias is a number"""
        cell_body_sum=np.sum(inputs*self.weights)+self.bias
        firing_rate=1.0/(1.0+math.exp(-cell_body_sum))
        return firing_rate
   

In other words, each neuron performs a dot product with the input and its weights, adds the bias and applies the non-linearity (or activation function), in this case the sigmoid $\sigma(x) = 1/(1+e^{-x})$
. We will go into more details about different activation functions at the end of this section.

### Single neuron as a linear classifier
A single neuron can be defined as linear classifier. e.g. binary softmax classifier , binary svm classifier.

**Regularization Interpretation**:The regularization loss in both SVM/Softmax cases could in this biological view be interpreted as gradual forgetting, since it would have the effect of driving all synaptic weights w towards zero after every parameter update.

### Commonly used activation functions
- Sigmoid
- tanH
- ReLU
- Leaky ReLU


## Sigmoid
**Formula** $\sigma(x) = 1 / (1 + e^{-x})$.  It takes a real-valued number and “squashes” it into range between 0 and 1. In particular, large negative numbers become 0 and large positive numbers become 1.In practice, the sigmoid non-linearity has recently fallen out of favor and it is rarely ever used. It has two major drawbacks:

- **Sigmoids saturate and kill gradients**:A very undesirable property of the sigmoid neuron is that when the neuron’s activation saturates at either tail of 0 or 1, the gradient at these regions is almost zero. Recall that during backpropagation, this (local) gradient will be multiplied to the gradient of this gate’s output for the whole objective. Therefore, if the local gradient is very small, it will effectively “kill” the gradient and almost no signal will flow through the neuron to its weights and recursively to its data. Additionally, one must pay extra caution when initializing the weights of sigmoid neurons to prevent saturation. For example, if the initial weights are too large then most neurons would become saturated and the network will barely learn.

- **Sigmoid outputs are not zero-centered**: This is undesirable since neurons in later layers of processing in a Neural Network (more on this soon) would be receiving data that is not zero-centered. This has implications on the dynamics during gradient descent, because if the data coming into a neuron is always positive e.g. $x>0$
 elementwise in $f=w^Tx+b$, then the gradient on the weights w will during backpropagation become either all be positive, or all negative (depending on the gradient of the whole expression f
). This could introduce undesirable zig-zagging dynamics in the gradient updates for the weights. However, notice that once these gradients are added up across a batch of data the final update for the weights can have variable signs, somewhat mitigating this issue. Therefore, this is an inconvenience but it has less severe consequences compared to the saturated activation problem above.

![alt text](Images/st.PNG "sigmoid and tanH")

## Tanh
The tanh non-linearity is shown on the image above on the right. It squashes a real-valued number to the range [-1, 1]. Like the sigmoid neuron, its activations saturate, but unlike the sigmoid neuron its output is zero-centered. Therefore, in practice the tanh non-linearity is always preferred to the sigmoid nonlinearity. Also note that the tanh neuron is simply a scaled sigmoid neuron, in particular the following holds: $tanh(x)=2σ(2x)−1$
.

## ReLU
The Rectified Linear Unit has become very popular in the last few years. It computes the function $f(x)=\max(0,x)$. In other words, the activation is simply thresholded at zero (see image above on the left). There are several pros and cons to using the ReLUs:

 **Pros**:
  - greatly accelearate (e.g. a factor of 6 in [Krizhevsky et al.](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf))the convergence of stochastic gradient descent compared to the sigmoid/tanh functions. It is argued that this is due to its linear, non-saturating form.
  - Compared to tanh/sigmoid neurons that involve expensive operations (exponentials, etc.), the ReLU can be implemented by simply thresholding a matrix of activations at zero.
  
 **Cons**:
  -  ReLU units can be fragile during training and can “die”. For example, a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on.
       - solutions: With a proper setting of the learning rate this is less frequently an issue.
    

![alt text](Images/rlr.PNG "ReLU")

## Leaky ReLU
Leaky ReLUs are one attempt to fix the “dying ReLU” problem. Instead of the function being zero when x < 0, a leaky ReLU will instead have a small positive slope (of 0.01, or so). That is, the function computes $f(x) = \mathbb{1}(x < 0) (\alpha x) + \mathbb{1}(x>=0) (x)$

## Maxout
Other types of units have been proposed that do not have the functional form $f(w^Tx + b)$ where a non-linearity is applied on the dot product between the weights and the data. One relatively popular choice is the Maxout neuron (introduced recently by [Goodfellow et al](https://arxiv.org/abs/1302.4389).) that generalizes the ReLU and its leaky version. The Maxout neuron computes the function  $\max(w_1^Tx+b_1, w_2^Tx + b_2)$
. Notice that both ReLU and Leaky ReLU are a special case of this form (for example, for ReLU we have ($w1,b1=0$). The Maxout neuron therefore enjoys all the benefits of a ReLU unit (linear regime of operation, no saturation) and does not have its drawbacks (dying ReLU). However, unlike the ReLU neurons it doubles the number of parameters for every single neuron, leading to a high total number of parameters.

## Neural Network architectures
- Neural Networks as neurons in graphs.Neural Networks are modeled as collections of neurons that are connected in an acyclic graph.  Neural Network models are often organized into distinct layers of neurons. For regular neural networks, the most common layer type is the fully-connected layer in which neurons between two adjacent layers are fully pairwise connected, but neurons within a single layer share no connections. Below are two example Neural Network topologies that use a stack of fully-connected layers:

![alt text](Images/architecture.PNG "nn architecture")

#### Naming Conventions
- N-layer means hidden and output layers. e.g. 2 layer means one hidden layer and a output layer. 1 layer means no hidden layer or linear classifier. This type of network is also referred as 'Artificial Neural Network' (ANN) or myltilayer perceptron (MLE)

#### output layer: 
doesn't have activation function.(or you can think of them as having a linear identity activation function)

#### size of the neural networks:
In the above image(right) two hidden layers weith 4 neurons each. input size 3 and output neuron is 1. Therfore we can demostrate size of neural networks as follows:
- 4+4+1=9 (not counting the inputs)neurons,[3 x 4] + [4 x 4] + [4 x 1] = 12 + 16 + 4 = 32 weights and 4 + 4 + 1 = 9 biases, for a total of 41 learnable parameters.

### Example of feed forward computations
for the above casej(right image) lets declare the inputs and weights,

W1=[4 X 3] , b1=[4 X 1] ,**Here, every single neuron has its weights in a row of**, so matrix vector multiplication in the first hidden layer is np.dot(W1,x) . W2 is [4 X 4] and b2=[4 X 1] and W3 will be [1 X 4] for the last output layer. Therefore full forward pass will be three matrix vector multiplications, interwoven with the applications of activation functions.

In [ ]:
# forward-pass of a 3-layer neural network:
f = lambda x: 1.0/(1.0 + np.exp(-x)) # activation function (use sigmoid)
x = np.random.randn(3, 1) # random input vector of three numbers (3x1)
h1 = f(np.dot(W1, x) + b1) # calculate first hidden layer activations (4x1)
h2 = f(np.dot(W2, h1) + b2) # calculate second hidden layer activations (4x1)
out = np.dot(W3, h2) + b3 # output neuron (1x1)

### layer size
 In practice it is often the case that 3-layer neural networks will outperform 2-layer nets, but going even deeper (4,5,6-layer) rarely helps much more. This is in stark contrast to Convolutional Networks, where depth has been found to be an extremely important component for a good recognition system (e.g. on order of 10 learnable layers). One argument for this observation is that images contain hierarchical structure (e.g. faces are made up of eyes, which are made up of edges, etc.), so several layers of processing make intuitive sense for this data domain.

### Setting number of layers and their sizes
**Q**: what should be the size of hidden layers for a practical problem? one?, two? three?.....
First, note that as we increase the size and number of layers in a Neural Network, the capacity of the network increases.suppose we had a binary classification problem in two dimensions. We could train three separate neural networks, each with one hidden layer of some size and obtain the following classifiers:

![alt text](Images/nn-layer.PNG "hidden layer size")Larger Neural Networks can represent more complicated functions. The data are shown as circles colored by their class, and the decision regions by a trained neural network are shown underneath. You can play with these examples in this [ConvNetsJS demo](https://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html).

In the diagram above, we can see that Neural Networks with more neurons can express more complicated functions. However, this is both a blessing (since we can learn to classify more complicated data) and a curse (since it is easier to overfit the training data). Overfitting occurs when a model with high capacity fits the noise in the data instead of the (assumed) underlying relationship. For example, the model with 20 hidden neurons fits all the training data but at the cost of segmenting the space into many disjoint red and green decision regions. The model with 3 hidden neurons only has the representational power to classify the data in broad strokes. It models the data as two blobs and interprets the few red points inside the green cluster as outliers (noise). In practice, this could lead to better generalization on the test set

**Recommendation**
- smaller neural networks can be preferred if the data is not complex enough to prevent overfitting. However, this is incorrect - there are many other preferred ways to prevent overfitting in Neural Networks that we will discuss later (such as L2 regularization, dropout, input noise). In practice, it is always better to use these methods to control overfitting instead of the number of neurons.

**Q** why should we use larger number of neurons in the hidden units?
Ans: Loss function of smaller networks have few local minima and many of these are easy to converge which are bad. To put it another way the loss is higher. On the other hand bneural networks contain significantly more local minima, but these minima turn out to be much better in terms of their actual loss.


**the regularization strength is the preferred way to control the overfitting of a neural network. We can look at the results achieved by three different settings:**
![alt text](Images/reg.PNG "regularization effect") The effects of regularization strength: Each neural network above has 20 hidden neurons, but changing the regularization strength makes its final decision regions smoother with a higher regularization. You can play with these examples in this [ConvNetsJS demo](https://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html)

### Conclusion:
The takeaway is that you should not be using smaller networks because you are afraid of overfitting. Instead, you should use as big of a neural network as your computational budget allows, and use other regularization techniques to control overfitting.